In [1]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
import vectorbtpro as vbt
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time
import helpers as pth
from numba import njit
import talib

theme = "light"
vbt.settings.set_theme(theme)
# optional
# vbt.settings["plotting"]["layout"]["width"] = 640
# vbt.settings["plotting"]["layout"]["height"] = 320

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12, 7)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]
plt.style.use("classic" if theme == "light" else "dark_background")

if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

In [2]:
start = "2024-10-01"
end = "2024-10-10"
symbols = ["MES"]
chunk_size = 2

chunks = pth.get_date_chunks(start, end, chunk_size=1)
chunks

[('2024-10-01', '2024-10-02'),
 ('2024-10-02', '2024-10-03'),
 ('2024-10-03', '2024-10-04'),
 ('2024-10-04', '2024-10-05'),
 ('2024-10-05', '2024-10-06'),
 ('2024-10-06', '2024-10-07'),
 ('2024-10-07', '2024-10-08'),
 ('2024-10-08', '2024-10-09'),
 ('2024-10-09', '2024-10-10')]

In [3]:
chunk_start, chunk_end = chunks[0][0], chunks[1][1]

df = pth.SierraChartData.pull(symbols, start=chunk_start, end=chunk_end)
df = df.resample("1min")
df.shape

(2880, 1)

### Simple stop loss

In [25]:
from collections import namedtuple


Memory = namedtuple("Memory", ["counter"])
init_memory = lambda target_shape: Memory(
    counter=np.full(target_shape[1], np.nan)
)


# @njit(nogil=True)
def signal_func(c, m):
    if c.i % 10 == 0 and not vbt.pf_nb.in_long_position_nb(c):
        return pth.LONG_ENTER()

    if vbt.pf_nb.in_long_position_nb(c):
        sl_info = c.last_sl_info[c.col]
        position =  c.last_pos_info[c.col]
        if not vbt.pf_nb.is_stop_info_active_nb(sl_info):
            entry_price = position['entry_price']
            # stop 1
            vbt.pf_nb.set_sl_info_nb(
                sl_info,
                init_price=entry_price,
                init_idx=c.i,
                stop=entry_price - 0.5,
                delta_format=vbt.pf_enums.DeltaFormat.Target,
                exit_size_type=vbt.pf_enums.SizeType.Amount,
                exit_size=position['size'] // 2,
            )
            # stop 2
            vbt.pf_nb.set_sl_info_nb(
                sl_info,
                init_price=entry_price,
                init_idx=c.i,
                stop=entry_price - 0.9,
                delta_format=vbt.pf_enums.DeltaFormat.Target,
                exit_size_type=vbt.pf_enums.SizeType.Amount,
                exit_size=position['size'] // 2,
            )

    return pth.NO_SIGNAL()


pf = vbt.Portfolio.from_signals(
    df.get('Close').iloc[:100],
    signal_func_nb=signal_func,
    signal_args=(
        vbt.RepFunc(init_memory),
    ),
    size=1.0,
    size_granularity=1,
    size_type=vbt.pf_enums.SizeType.Percent,
    init_cash=100000,
    jitted=False,
)
pf.plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': 'd2eb2a19-84ff-425a-bb37-b469640541d4',
              'x': array([datetime.datetime(2024, 10, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 2, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 3, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 4, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 5, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 6, tzinfo=datetime.timezone.utc),
                          datetime.date

In [26]:
pf.plot_trades()

FigureWidget({
    'data': [{'line': {'color': '#1f77b4'},
              'mode': 'lines',
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '4e1ada5e-ba68-4675-9d10-85c6f4512503',
              'x': array([datetime.datetime(2024, 10, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 2, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 3, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 4, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 5, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 6, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 7, tzinfo=datetime.timezone.utc),

In [27]:
pf.orders.readable

,Order Id,Column,Signal Index,Creation Index,Fill Index,Size,Price,Fees,Side,Type,Stop Type
0,0,MES,2024-10-01 00:00:00+00:00,2024-10-01 00:00:00+00:00,2024-10-01 00:00:00+00:00,16.0,5929.0,0.0,Buy,Market,None
1,1,MES,2024-10-01 00:01:00+00:00,2024-10-01 00:01:00+00:00,2024-10-01 00:01:00+00:00,8.0,5928.1,0.0,Sell,Market,SL
2,2,MES,2024-10-01 00:02:00+00:00,2024-10-01 00:02:00+00:00,2024-10-01 00:02:00+00:00,8.0,5928.1,0.0,Sell,Market,SL
3,3,MES,2024-10-01 00:10:00+00:00,2024-10-01 00:10:00+00:00,2024-10-01 00:10:00+00:00,16.0,5928.0,0.0,Buy,Market,None
4,4,MES,2024-10-01 00:11:00+00:00,2024-10-01 00:26:00+00:00,2024-10-01 00:26:00+00:00,8.0,5927.1,0.0,Sell,Market,SL
5,5,MES,2024-10-01 00:27:00+00:00,2024-10-01 00:27:00+00:00,2024-10-01 00:27:00+00:00,8.0,5927.1,0.0,Sell,Market,SL
6,6,MES,2024-10-01 00:30:00+00:00,2024-10-01 00:30:00+00:00,2024-10-01 00:30:00+00:00,16.0,5927.0,0.0,Buy,Market,None


In [28]:
pf.positions.readable

,Position Id,Column,Size,Entry Order Id,Entry Index,Avg Entry Price,Entry Fees,Exit Order Id,Exit Index,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status
0,0,MES,16.0,0,2024-10-01 00:00:00+00:00,5929.0,0.0,2,2024-10-01 00:02:00+00:00,5928.1,0.0,-14.4,-0.000152,Long,Closed
1,1,MES,16.0,3,2024-10-01 00:10:00+00:00,5928.0,0.0,5,2024-10-01 00:27:00+00:00,5927.1,0.0,-14.4,-0.000152,Long,Closed
2,2,MES,16.0,6,2024-10-01 00:30:00+00:00,5927.0,0.0,-1,2024-10-01 01:39:00+00:00,5933.0,0.0,96.0,0.001012,Long,Open


### Stop and Target

In [32]:
from collections import namedtuple


Memory = namedtuple("Memory", ["counter"])
init_memory = lambda target_shape: Memory(
    counter=np.full(target_shape[1], np.nan)
)


# @njit(nogil=True)
def signal_func(c, m):
    if c.i % 10 == 0:
        return pth.LONG_ENTER()

    position =  c.last_pos_info[c.col]
    if position['status'] == vbt.pf_enums.TradeStatus.Open:
        sl_info = c.last_sl_info[c.col]
        tp_info = c.last_tp_info[c.col]
        if not vbt.pf_nb.is_stop_info_active_nb(sl_info):
            if vbt.pf_nb.in_long_position_nb(c):
                entry_price = position['entry_price']
                # stop
                vbt.pf_nb.set_sl_info_nb(
                    sl_info,
                    init_idx=c.i,
                    init_price=entry_price,
                    stop=entry_price - 1.0,
                    delta_format=vbt.pf_enums.DeltaFormat.Target,
                    exit_size_type=vbt.pf_enums.SizeType.Amount,
                    exit_size=position['size'],
                )
                # target
                vbt.pf_nb.set_tp_info_nb(
                    tp_info,
                    init_idx=c.i,
                    init_price=entry_price,
                    stop=entry_price + 2.0,
                    delta_format=vbt.pf_enums.DeltaFormat.Target,
                    exit_size_type=vbt.pf_enums.SizeType.Amount,
                    exit_size=position['size'],
                )

    return pth.NO_SIGNAL()


pf = vbt.Portfolio.from_signals(
    df.get('Close').iloc[:100],
    signal_func_nb=signal_func,
    signal_args=(
        vbt.RepFunc(init_memory),
    ),
    size=1.0,
    size_granularity=1,
    size_type=vbt.pf_enums.SizeType.Percent,
    init_cash=100000,
    jitted=False,
)
pf.plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '2906bd14-21e6-4506-a1b5-8f0b20825085',
              'x': array([datetime.datetime(2024, 10, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 2, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 3, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 4, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 5, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 6, tzinfo=datetime.timezone.utc),
                          datetime.date

In [33]:
pf.plot_trades()

FigureWidget({
    'data': [{'line': {'color': '#1f77b4'},
              'mode': 'lines',
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f05aa1e4-b874-4455-9155-2a8a337c369b',
              'x': array([datetime.datetime(2024, 10, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 2, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 3, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 4, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 5, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 6, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 7, tzinfo=datetime.timezone.utc),

In [34]:
pf.orders.readable

,Order Id,Column,Signal Index,Creation Index,Fill Index,Size,Price,Fees,Side,Type,Stop Type
0,0,MES,2024-10-01 00:00:00+00:00,2024-10-01 00:00:00+00:00,2024-10-01 00:00:00+00:00,16.0,5929.00,0.0,Buy,Market,None
1,1,MES,2024-10-01 00:01:00+00:00,2024-10-01 00:01:00+00:00,2024-10-01 00:01:00+00:00,16.0,5928.00,0.0,Sell,Market,SL
2,2,MES,2024-10-01 00:10:00+00:00,2024-10-01 00:10:00+00:00,2024-10-01 00:10:00+00:00,16.0,5928.00,0.0,Buy,Market,None
3,3,MES,2024-10-01 00:11:00+00:00,2024-10-01 00:26:00+00:00,2024-10-01 00:26:00+00:00,16.0,5927.00,0.0,Sell,Market,SL
4,4,MES,2024-10-01 00:30:00+00:00,2024-10-01 00:30:00+00:00,2024-10-01 00:30:00+00:00,16.0,5927.00,0.0,Buy,Market,None
5,5,MES,2024-10-01 00:31:00+00:00,2024-10-01 00:54:00+00:00,2024-10-01 00:54:00+00:00,16.0,5929.00,0.0,Sell,Market,TP
6,6,MES,2024-10-01 01:00:00+00:00,2024-10-01 01:00:00+00:00,2024-10-01 01:00:00+00:00,16.0,5932.75,0.0,Buy,Market,None
7,7,MES,2024-10-01 01:01:00+00:00,2024-10-01 01:01:00+00:00,2024-10-01 01:01:00+00:00,16.0,5931.75,0.0,Sell,Market,SL
8,8,MES,2024-10-01 01:10:00+00:00,2024-10-01 01:10:00+00:00,2024-10-01 01:10:00+00:00,16.0,5932.75,0.0,Buy,Market,None
9,9,MES,2024-10-01 01:11:00+00:00,2024-10-01 01:14:00+00:00,2024-10-01 01:14:00+00:00,16.0,5931.75,0.0,Sell,Market,SL


In [35]:
pf.positions.readable

,Position Id,Column,Size,Entry Order Id,Entry Index,Avg Entry Price,Entry Fees,Exit Order Id,Exit Index,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status
0,0,MES,16.0,0,2024-10-01 00:00:00+00:00,5929.00,0.0,1,2024-10-01 00:01:00+00:00,5928.00,0.0,-16.0,-0.000169,Long,Closed
1,1,MES,16.0,2,2024-10-01 00:10:00+00:00,5928.00,0.0,3,2024-10-01 00:26:00+00:00,5927.00,0.0,-16.0,-0.000169,Long,Closed
2,2,MES,16.0,4,2024-10-01 00:30:00+00:00,5927.00,0.0,5,2024-10-01 00:54:00+00:00,5929.00,0.0,32.0,0.000337,Long,Closed
3,3,MES,16.0,6,2024-10-01 01:00:00+00:00,5932.75,0.0,7,2024-10-01 01:01:00+00:00,5931.75,0.0,-16.0,-0.000169,Long,Closed
4,4,MES,16.0,8,2024-10-01 01:10:00+00:00,5932.75,0.0,9,2024-10-01 01:14:00+00:00,5931.75,0.0,-16.0,-0.000169,Long,Closed
5,5,MES,16.0,10,2024-10-01 01:20:00+00:00,5928.75,0.0,11,2024-10-01 01:21:00+00:00,5927.75,0.0,-16.0,-0.000169,Long,Closed
6,6,MES,16.0,12,2024-10-01 01:30:00+00:00,5932.00,0.0,-1,2024-10-01 01:39:00+00:00,5933.00,0.0,16.0,0.000169,Long,Open


### Time of Day entry / exit signals

In [54]:
from collections import namedtuple


Memory = namedtuple("Memory", ["counter"])
init_memory = lambda target_shape: Memory(
    counter=np.full(target_shape[1], np.nan)
)


# @njit(nogil=True)
def signal_func(c, m):
    timestamp = c.index[c.i]
    hour = vbt.dt_nb.hour_nb(timestamp)
    position =  c.last_pos_info[c.col]
    in_position = position['status'] == vbt.pf_enums.TradeStatus.Open
    if hour % 3 == 0 and not in_position:
        return pth.SHORT_ENTER()
    if hour % 3 != 0 and in_position:
        return pth.SHORT_EXIT()
    return pth.NO_SIGNAL()


pf = vbt.Portfolio.from_signals(
    df.get('Close'),
    signal_func_nb=signal_func,
    signal_args=(
        vbt.RepFunc(init_memory),
    ),
    size=1.0,
    size_granularity=1,
    size_type=vbt.pf_enums.SizeType.Percent,
    init_cash=100000,
    jitted=False,
)
pf.plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '7cdd737a-80a3-49d3-8e99-5d3265cc1beb',
              'x': array([datetime.datetime(2024, 10, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 10, 2, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 2, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 2, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([100000., 100000., 100000., ..., 100000., 100000., 10000

In [55]:
pf.plot_trades()

FigureWidget({
    'data': [{'line': {'color': '#1f77b4'},
              'mode': 'lines',
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '2788d5bf-a2d6-45ca-b3e4-374e296db29a',
              'x': array([datetime.datetime(2024, 10, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 10, 2, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 2, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 2, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([5929.  , 5928.  , 5927.5 , ..., 5891.25, 5890.75, 5890.5 ])},
             {'customdata': array([['0', '0', '

In [45]:
pf.orders.readable

,Order Id,Column,Signal Index,Creation Index,Fill Index,Size,Price,Fees,Side,Type,Stop Type
0,0,MES,2024-10-01 01:00:00+00:00,2024-10-01 01:00:00+00:00,2024-10-01 01:00:00+00:00,16.0,5932.75,0.0,Buy,Market,None
1,1,MES,2024-10-01 05:00:00+00:00,2024-10-01 05:00:00+00:00,2024-10-01 05:00:00+00:00,16.0,5936.75,0.0,Sell,Market,None
2,2,MES,2024-10-02 01:00:00+00:00,2024-10-02 01:00:00+00:00,2024-10-02 01:00:00+00:00,17.0,5875.00,0.0,Buy,Market,None
3,3,MES,2024-10-02 05:00:00+00:00,2024-10-02 05:00:00+00:00,2024-10-02 05:00:00+00:00,17.0,5883.25,0.0,Sell,Market,None


In [57]:
vbt.pf_nb.order_nb??

Signature:      
vbt.pf_nb.order_nb(
    size: float = inf,
    price: float = inf,
    size_type: int = 0,
    direction: int = 2,
    fees: float = 0.0,
    fixed_fees: float = 0.0,
    slippage: float = 0.0,
    min_size: float = nan,
    max_size: float = nan,
    size_granularity: float = nan,
    leverage: float = 1.0,
    leverage_mode: int = 0,
    reject_prob: float = 0.0,
    price_area_vio_mode: int = 0,
    allow_partial: bool = True,
    raise_reject: bool = False,
    log: bool = False,
) -> vectorbtpro.portfolio.enums.Order
Call signature:  vbt.pf_nb.order_nb(*args, **kwargs)
Type:            CPUDispatcher
String form:     CPUDispatcher(<function order_nb at 0x000001BF05F03100>)
File:            c:\users\spask\miniconda3\lib\site-packages\vectorbtpro\portfolio\nb\core.py
Source:         
@register_jitted(cache=True)
def order_nb(
    size: float = np.inf,
    price: float = np.inf,
    size_type: int = SizeType.Amount,
    direction: int = Direction.Both,
    fees: float